## Package Imports

In [1]:
import pandas as pd
import numpy as np
import re
import json
import boto3
import s3fs
from azure.storage.blob import BlobServiceClient, generate_account_sas, ResourceTypes, AccountSasPermissions
import tables
from datetime import datetime, timedelta


## Read dataset from public cloud storage

In [2]:
# def read_prefix_to_df(prefix):
#     s3 = boto3.resource('s3')
#     bucket = s3.Bucket('test-bucket')
#     prefix_objs = bucket.objects.filter(Prefix=prefix)
#     prefix_df = []
#     for obj in prefix_objs:
#         key = obj.key
#         body = obj.get()['Body'].read()
#         df = pd.DataFrame(body)
#         prefix_df.append(df)
#     return pd.concat(prefix_df)

In [3]:
data_location = input("From which location we are reading data?").lower()
if data_location in ["aws","s3","amazon"]:
    AWS_S3_BUCKET = input("Your bucket name please")
    file_name = input("Your full file name please (with extension)")
    if "csv" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_csv(f"s3://{AWS_S3_BUCKET}/de/{file_name[0]}.csv")
    elif "parquet" or "pqt" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_parquet(f"s3://{AWS_S3_BUCKET}/de/{file_name[0]}.parquet")
    else:
        print("Unavailable file format")
# Not confirmed yet to be developed by reading mechanisim
elif data_location in ["azure","blob"]:
    ACCOUNT_NAME = input("Your Azure account name please")
    STORAGE_ACCOUNT_NAME = input("Your Azure account storage name please")
    ACCOUNT_KEY = input("Your Azure account key please")
    file_name = input("Your full file name please (with extension)")
    blob_name = input("Your blob name please")
    container_name = input("Your container name please")
    # Creating temp sas token
    sas_token = generate_account_sas(
    account_name="{STORAGE_ACCOUNT_NAME}",
    account_key="{ACCOUNT_KEY}",
    resource_types=ResourceTypes(service=True),
    permission=AccountSasPermissions(read=True),
    expiry=datetime.utcnow() + timedelta(hours=1)
    )
    # Creating service client var 
    blob_service_client = BlobServiceClient(account_url=f"https://{ACCOUNT_NAME}.blob.core.windows.net"
                                            , credential=sas_token)
    blob_service_client.get_blob_to_path(container_name,blob_name,file_name)
    if "csv" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_csv(f"{file_name}")
    elif "parquet" or "pqt" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_parquet(f"{file_name}")
    else:
        print("Unavailable file format")
else:
    print("no connector available from given file source location")



## General Details over Dataset

### Head rows of Dataset

In [4]:
head_df = df.head(3)
display(head_df)

,adjective,category,created_at,id,modifier,name,price
0,fuzzy,contraption,2014-01-15 21:36:09,2512.0,carrying_case,fuzzy contraption carrying_case,150.0
1,NaN,instrument,2013-05-14 05:20:50,482.0,refill,instrument refill,35.2
2,industrial-strength,module,2014-02-04 19:28:32,2446.0,NaN,industrial-strength module,300.0


### Column data types

In [5]:
col_types = df.dtypes
df_col_types = pd.DataFrame([col_types.values],columns=[df.columns])
display(df_col_types)

,adjective,category,created_at,id,modifier,name,price
0,object,object,object,float64,object,object,float64


### Size of Dataset

In [6]:
shape_df = df.shape
df_shape = pd.DataFrame([shape_df],columns = ["rows","cols"])
display(df_shape)

,rows,cols
0,2198,7


### Non numerical columns

In [7]:
describe_non_numerical = df.describe(exclude=[np.number])
display(describe_non_numerical)

,adjective,category,created_at,modifier,name
count,1926,2198,2198,1432,2198
unique,13,10,2198,9,1011
top,reflective,instrument,2014-01-15 21:36:09,storage_unit,apparatus
freq,168,239,1,186,13


### All columns general stats

In [8]:
describe_all_cols = df.describe(include='all')
display(describe_all_cols)

,adjective,category,created_at,id,modifier,name,price
count,1926,2198,2198,2198.000000,1432,2198,2198.000000
unique,13,10,2198,NaN,9,1011,NaN
top,reflective,instrument,2014-01-15 21:36:09,NaN,storage_unit,apparatus,NaN
freq,168,239,1,NaN,186,13,NaN
mean,NaN,NaN,NaN,1991.293904,NaN,NaN,98.157420
std,NaN,NaN,NaN,1163.964428,NaN,NaN,156.059937
min,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
25%,NaN,NaN,NaN,981.250000,NaN,NaN,13.750000
50%,NaN,NaN,NaN,2008.000000,NaN,NaN,42.000000
75%,NaN,NaN,NaN,2994.500000,NaN,NaN,115.700000


### Detect candidate key columns (Unique columns)

In [9]:
unique_val_holding_cols = [i for i in df.columns if len(df[i]) == len(pd.unique(df[i]))]
df_unique = pd.DataFrame(unique_val_holding_cols,columns = ["unique_val_holding_cols"])
display(df_unique)

,unique_val_holding_cols
0,created_at
1,id


### Null value counts per column

In [10]:
null_val_holding_cols = [df[i].isna().sum() for i in df.columns]
df_null = pd.DataFrame([null_val_holding_cols],columns = df.columns)
display(df_null)

,adjective,category,created_at,id,modifier,name,price
0,272,0,0,0,766,0,0


### Detect JSON Column by col values

In [11]:
# Detect JSON Column by col name
# col_with_dot_char = [df[i].name for i in df.columns if "." in df[i].name]
# print(col_with_dot_char)

col_with_json_val = [df[i].name for i in df.columns if "{" in str(df[i].iloc[0])]
df_json_cols = pd.DataFrame(col_with_json_val,columns = ["col_with_json_val"])
display(df_json_cols)

,col_with_json_val


### Detect XML Column by col values

In [12]:
col_with_xml_val = [df[i].name for i in df.columns if "<" in str(df[i].iloc[0])]
df_xml_cols = pd.DataFrame(col_with_xml_val,columns = ["col_with_xml_val"])
display(df_xml_cols)

,col_with_xml_val


### Detect Date-Time Columns

In [13]:
#Regex pattern detect date-time columns
pattern_dt = re.compile(r"[0-9]{4}.[0-9]{2}.[0-9]{2}\s[0-9]{2}.[0-9]{2}.[0-9]{2}(\.[0-9]{1,3})?", re.IGNORECASE)
#Loop through columns detect date-time columns
col_date_time = [df[i].name for i in df.columns if pattern_dt.match(str(df[i].iloc[0]))]
df_date_time_cols = pd.DataFrame(col_date_time,columns = ["col_date_time"])
display(df_date_time_cols)

,col_date_time
0,created_at


### Detect Date Columns

In [14]:
#Regex pattern for date columns
pattern_d = re.compile(r"[0-9]{4}.[0-9]{2}.[0-9]{2}", re.IGNORECASE)
#Loop through columns detect date columns
col_date_time = [df[i].name for i in df.columns if pattern_dt.match(str(df[i].iloc[0]))]
df_date_time_cols = pd.DataFrame(col_date_time,columns = ["col_date_time"])
display(df_date_time_cols)

,col_date_time
0,created_at
